<a href="https://colab.research.google.com/github/wooohun/BERT-Summarizer/blob/main/BART_Abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies
!pip install transformers
!pip install torch
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import torch
import nltk
import evaluate
from datasets import load_dataset, load_metric
from evaluate import evaluator

In [3]:
# install kaggle
!pip install -q kaggle
!mkdir ~/.kaggle

# get kaggle api token from account -> API -> create new API Token
# move kaggle api token to kaggle folder
!cp -v kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
# download dataset
# !chmod 600 /root/.kaggle/kaggl
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail
!unzip newspaper-text-summarization-cnn-dailymail

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.9/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.9/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open newspaper-text-summarization-cnn-dailymail, newspaper-text-summarization-cnn-dailymail.zip or newspaper-text-summarization-cnn-dailymail.ZIP.


In [5]:
# grab datasets
dataset = load_dataset('cnn_dailymail', '3.0.0')
# formatted as:
# DatasetDict{
#   train: Dataset{
#     features:[]
#     num_rows: int 
#   }
#   test: {}
#   validation: {}  
# }

  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
from transformers import BartForConditionalGeneration, BartTokenizerFast, BartConfig

# configure Bart model to use gradient checkpointing
config = BartConfig.from_pretrained('facebook/bart-base', gradient_checkpointing=True)
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base', config=config)



In [12]:
def preprocessing(dataset):
  inputs = [article for article in dataset['article']]
  tokenized_inputs = tokenizer(inputs, max_length = 1024, truncation=True)

  # set tokenizer to encode labels
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(dataset['highlights'], max_length = 128, truncation=True)

  tokenized_inputs['labels'] = labels['input_ids']
  return tokenized_inputs

In [13]:
# process all datasets in batches using fast tokenizer for efficiency
processed_dataset = dataset.map(preprocessing, batched = True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [14]:
tokenizer.batch_decode(processed_dataset['train']['labels'][0], skip_special_tokens=True)

['',
 'Harry',
 ' Potter',
 ' star',
 ' Daniel',
 ' Rad',
 'cliffe',
 ' gets',
 ' £',
 '20',
 'M',
 ' fortune',
 ' as',
 ' he',
 ' turns',
 ' 18',
 ' Monday',
 '.',
 '\n',
 'Young',
 ' actor',
 ' says',
 ' he',
 ' has',
 ' no',
 ' plans',
 ' to',
 ' f',
 'rit',
 'ter',
 ' his',
 ' cash',
 ' away',
 '.',
 '\n',
 'Rad',
 'cliffe',
 "'s",
 ' earnings',
 ' from',
 ' first',
 ' five',
 ' Potter',
 ' films',
 ' have',
 ' been',
 ' held',
 ' in',
 ' trust',
 ' fund',
 '.',
 '']

In [15]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

In [16]:
# create training args
batch_size = 4
num_train_epochs = 2
logging_steps = len(processed_dataset['train'])
gradient_accumulation_steps = 8  # accumulate gradients over 8 steps

args = Seq2SeqTrainingArguments(
    output_dir = "facebook-bart-base-finetuned-cnn-dailymail",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    gradient_accumulation_steps=gradient_accumulation_steps  # add gradient accumulation parameter
)


In [17]:
# install metrics
!pip install bert_score
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=68a266a25e64104f96d66d80e96985e768ccc1c79cd5d79a70b2f2505929166c
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [18]:
# load metrics
rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')

In [19]:
import numpy as np

# metric computation function to pass into trainer object
def metric_compute(predicted):
  predictions, labels = predicted
  # decode predictions, labels for metric computation
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # batch_decode returns list of tokens, use nltk to convert list of tokens to list of sentences
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

  # # compute bert_score
  # bert_score_res = bert_score.compute(
  #     predictions = decoded_preds, references = decoded_labels, use_stemmer=True
  # )
  rouge_res = rouge.compute(
      predictions = decoded_preds, references = decoded_labels, use_stemmer=True
  )
  # result = {key: value.mid.fmeasure * 100 for key, value in bert_score_res.items()}
  # for key, value in rouge_res.items():
  #   result[key] = value.mid.fmeasure * 100
  result = {key: value.mid.fmeasure * 100 for key, value in rouge_res.items()}
  
  return {key: round(val, 4) for key, val in result.items()}

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [21]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metric_compute
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
!pip install pynvml nvidia-ml-py3

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()